In [13]:
from Input_function import import_data1
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sp

In [14]:
path="D:\\OneDrive - Indian Institute of Science\\4th Sem\\NSP\\NSP-Grant-Proposal\\MATLAB-COde\\forPython.mat"
[train_attributes,train_labels,test_attributes,test_labels]=import_data1(path)
#jupyter nbextension enable varInspector/main
